In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv(r"C:\Users\Sejal Hanmante\Downloads\phiusiil+phishing+url+dataset\PhiUSIIL_Phishing_URL_Dataset.csv")
df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


In [3]:
df.drop(["FILENAME","URL","Domain","Title"], axis=1, inplace=True)

In [4]:
df.select_dtypes("object")

,TLD
0,com
1,de
2,uk
3,com
4,org
...,...
235790,com
235791,uk
235792,be
235793,dev


In [5]:
a = df.groupby("TLD")["label"].count().nlargest(50)
a

TLD
com     112554
org      18793
net       7097
app       6508
uk        6395
co        5422
io        4201
de        3996
ru        3875
au        2979
dev       2345
top       2329
jp        2219
it        1887
edu       1861
fr        1858
br        1846
nl        1727
ca        1614
info      1566
site      1482
xyz       1473
link      1463
in        1379
pl        1339
gov       1276
cf        1203
ga        1111
us        1108
me        1006
ml         995
cn         993
eu         980
id         802
es         705
se         704
be         696
nz         614
cz         587
ch         560
ro         549
at         517
club       517
za         508
gr         500
mx         498
gq         493
page       490
ie         468
biz        429
Name: label, dtype: int64

In [6]:
b = df.groupby(["TLD","label"])["IsDomainIP"].count().nsmallest(10)
b

TLD       label
100       0        1
107       0        1
11        0        1
110       0        1
111       0        1
121       0        1
125       0        1
126       0        1
126:8080  0        1
133:8080  0        1
Name: IsDomainIP, dtype: int64

In [7]:
X = df.drop(["label","TLD"],axis=1)
y = df["label"]


### Feature selection for classification models
1.Information Gain

In [8]:
from sklearn.feature_selection import mutual_info_classif
import plotly.express as px
%matplotlib inline 
importances = mutual_info_classif(X,y)
feat_importance = pd.Series(importances,X.columns)
px.bar(feat_importance,orientation="h")

In [9]:
# Calculate mutual information for each feature
importances = mutual_info_classif(X, y)

# Create a Series with feature names and importance scores
feat_importance = pd.Series(importances, index=X.columns)

# Define a threshold for information gain (adjust as needed)
threshold = 0.4 # You can experiment with different values

# Select features with information gain above the threshold
selected_features = feat_importance[feat_importance >= threshold].index.tolist()

# Filter the original data using the selected features
X_filtered = X[selected_features]

# Now you can use X_filtered for further analysis or modeling
print("Selected features:", selected_features)

Selected features: ['URLSimilarityIndex', 'LineOfCode', 'LargestLineLength', 'HasSocialNet', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfExternalRef']


2.Feature Selection using Correlation Coefficient

In [10]:
cor = X.corr()
cor
columns = np.full((cor.shape[0],), True, dtype=bool)
for i in range(cor.shape[0]):
    for j in range(i+1, cor.shape[0]):
        if cor.iloc[i,j] >= 0.7:
            if columns[j]:
                columns[j] = False

selected_columns = X.columns[columns]
selected_columns.shape

(42,)

3. Backward elimination

In [11]:
import statsmodels.formula.api as sm
from statsmodels.regression.linear_model import OLS

def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(numVars - 1, -1, -1):
        regressor_OLS = OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            if (regressor_OLS.pvalues[i].astype(float) == maxVar):
                x = np.delete(x, i, 1)
                columns = np.delete(columns, i)
    regressor_OLS.summary()
    return x, columns

In [12]:
SL = 0.01
data_modeled, selected_columns1 = backwardElimination(X.values,y.values,SL, selected_columns)

In [13]:
selected_columns1.shape

(41,)

#### On applying various feature selection techniques correlation and backward elemination technique gives us similar results . Hence , we will be training our models on the features selected by these two

4. Bidirectional Feature Selection

In [14]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X = df.drop(["label","TLD"],axis=1)
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=200)

# Initialize RFE with the model and specify the number of features to select
num_features_to_select = 2  # Specify the number of features you want to select
rfe = RFE(model, n_features_to_select=num_features_to_select)

# Fit RFE
rfe.fit(X_train, y_train)

# Check if X_train is a DataFrame
if isinstance(X_train, pd.DataFrame):
    # Get the selected feature names directly
    selected_features = X_train.columns[rfe.support_]
else:
    # Convert X_train to a DataFrame to get column names
    X_train_df = pd.DataFrame(X_train, columns=X.columns)
    selected_features = X_train_df.columns[rfe.support_]

# Print selected feature names
print(f"Selected features: {selected_features.tolist()}")

d:\Anaconda\envs\env1\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

d:\Anaconda\envs\env1\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

d:\Anaconda\envs\env1\lib\site-packages\sklearn\

Selected features: ['URLLength', 'IsHTTPS']


In [15]:
backward_X = X[selected_columns1] # contains columns selected by backward elimination
corr_X = X[selected_columns] # contains columns selected by correlation 

# One-Hot encoding for TLD column 

In [16]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder(sparse_output=False)

tld = pd.DataFrame(df["TLD"])

TLD_encoded = one.fit_transform(tld)
# Convert the result to a DataFrame with appropriate column names
TLD_encoded_df = pd.DataFrame(TLD_encoded, columns=one.get_feature_names_out(['TLD']))

print(TLD_encoded_df)

        TLD_100  TLD_101  TLD_103  TLD_106  TLD_107  TLD_108  TLD_11  TLD_110  \
0           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
1           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
2           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
3           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
4           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
...         ...      ...      ...      ...      ...      ...     ...      ...   
235790      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235791      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235792      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235793      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235794      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   

        TLD_111  TLD_116  .

In [17]:
backward_updated = pd.merge(backward_X,TLD_encoded_df,left_index=True, right_index=True)
backward_updated # contains encoded tld column

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,31,24,0,100.000000,1.000000,0.522907,0.061933,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,16,0,100.000000,0.666667,0.032650,0.050207,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,22,0,100.000000,0.866667,0.028555,0.064129,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26,19,0,100.000000,1.000000,0.522907,0.057606,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33,26,0,100.000000,1.000000,0.079963,0.059441,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,29,22,0,100.000000,1.000000,0.522907,0.058739,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235791,28,21,0,100.000000,0.785714,0.028555,0.053834,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235792,30,23,0,100.000000,1.000000,0.003319,0.063093,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235793,55,47,0,28.157537,0.465116,0.000961,0.050211,3,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
corr_updated = pd.merge(corr_X,TLD_encoded_df,left_index=True, right_index=True)
corr_updated # contains encoded tld column

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,31,24,0,100.000000,1.000000,0.522907,0.061933,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,16,0,100.000000,0.666667,0.032650,0.050207,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,22,0,100.000000,0.866667,0.028555,0.064129,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26,19,0,100.000000,1.000000,0.522907,0.057606,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33,26,0,100.000000,1.000000,0.079963,0.059441,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,29,22,0,100.000000,1.000000,0.522907,0.058739,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235791,28,21,0,100.000000,0.785714,0.028555,0.053834,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235792,30,23,0,100.000000,1.000000,0.003319,0.063093,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235793,55,47,0,28.157537,0.465116,0.000961,0.050211,3,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Checking correlation between TLD and Label column

### 1. Chi Square test- 
P-value: If the p-value is less than your significance level (e.g., 0.05), you can reject the null hypothesis and conclude that there is a significant association between TLD and label.

In [19]:

from scipy.stats import chi2_contingency

# Assuming df is your DataFrame
# Create a contingency table
contingency_table = pd.crosstab(df['TLD'], df['label'])

# Perform Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Results
print(f"Chi-Square value: {chi2}")
print(f"P-value: {p}")

Chi-Square value: 72412.58849663034
P-value: 0.0


### 2. Cramér's V
Cramér's V is a measure of association between two nominal variables, giving a value between 0 and 1, where 0 indicates no association and 1 indicates perfect association. A higher value (closer to 1) indicates a stronger association between TLD and label.

In [20]:

# Compute Cramér's V
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

# Apply to the contingency table
cramers_v_value = cramers_v(contingency_table.values)
print(f"Cramér's V: {cramers_v_value}")

Cramér's V: 0.5541658258060268


### 3. Point-Biserial Correlation (If label is binary)
To measure the strength of association between the binary label and the categorical TLD. The value ranges from -1 to 1, where values closer to 1 or -1 indicate a stronger correlation.

In [21]:
from scipy.stats import pointbiserialr
from sklearn.preprocessing import LabelEncoder

# Encode the TLD column
le = LabelEncoder()
encoded_TLD = le.fit_transform(df['TLD'])

# Calculate the point-biserial correlation
corr, p_value = pointbiserialr(df['label'], encoded_TLD)

print(f"Point-Biserial Correlation: {corr}")
print(f"P-value: {p_value}")

Point-Biserial Correlation: 0.03927092389732449
P-value: 3.942503622610107e-81


### The TLD column should be considered an important feature for any predictive modeling or decision-making process related to identifying phishing websites. The statistical tests (Chi-Square, Cramér's V) strongly support its relevance.

### Detecting outliers

In [22]:
import plotly .express as px

In [23]:
'''
for i in df.select_dtypes("number").columns:

    print(f"for column:{i}")
    fig=px.box(df,y=df[i],x=df['label'],title=f'Box Plot of {i} by Label')
    fig.show()'''

'\nfor i in df.select_dtypes("number").columns:\n\n    print(f"for column:{i}")\n    fig=px.box(df,y=df[i],x=df[\'label\'],title=f\'Box Plot of {i} by Label\')\n    fig.show()'

### Removing Outliers using IQR

In [24]:
'''def removeoutliers(df):
    
        q1=df.quantile(0.25)
        q3=df.quantile(0.75)
        iqr=q3-q1
        lb=q1-1.5*iqr
        ub=q3+ 1.5*iqr
        return df[(df>=lb)&(df<=ub)]
    
df_cleaned=pd.DataFrame()
for i in df.select_dtypes("number").columns:
        df_cleaned[i]=removeoutliers(backward_updated[i])
df_cleaned_backward_updated =df_cleaned.dropna()
print(df_cleaned)'''




'def removeoutliers(df):\n    \n        q1=df.quantile(0.25)\n        q3=df.quantile(0.75)\n        iqr=q3-q1\n        lb=q1-1.5*iqr\n        ub=q3+ 1.5*iqr\n        return df[(df>=lb)&(df<=ub)]\n    \ndf_cleaned=pd.DataFrame()\nfor i in df.select_dtypes("number").columns:\n        df_cleaned[i]=removeoutliers(backward_updated[i])\ndf_cleaned_backward_updated =df_cleaned.dropna()\nprint(df_cleaned)'

#### Outlier removal for backward updated

In [25]:
import pandas as pd

def replace_outliers(df):
    cleaned_df = df.copy()
    for column in df.select_dtypes(include='number').columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lb = q1 - 1.5 * iqr
        ub = q3 + 1.5 * iqr
        
        # Replace the outliers
        cleaned_df.loc[cleaned_df[column] < lb, column] = lb
        cleaned_df.loc[cleaned_df[column] > ub, column] = ub
    
    return cleaned_df

# Example usage
df_cleaned = replace_outliers(backward_updated)
df_cleaned_backward_updated = df_cleaned.dropna()

print(df_cleaned_backward_updated)


        URLLength  DomainLength  IsDomainIP  URLSimilarityIndex  \
0            31.0            24           0          100.000000   
1            23.0            16           0          100.000000   
2            29.0            22           0          100.000000   
3            26.0            19           0          100.000000   
4            33.0            26           0          100.000000   
...           ...           ...         ...                 ...   
235790       29.0            22           0          100.000000   
235791       28.0            21           0          100.000000   
235792       30.0            23           0          100.000000   
235793       50.5            36           0           28.157537   
235794       33.0            26           0          100.000000   

        CharContinuationRate  TLDLegitimateProb  URLCharProb  TLDLength  \
0                   1.000000           0.522907     0.061933        3.0   
1                   0.666667           0.0326

#### Outlier removal for correlation updated

In [26]:
df_cleaned1 = replace_outliers(corr_updated)
df_cleaned_corr_updated = df_cleaned1.dropna()

df_cleaned_corr_updated

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,31.0,24,0,100.000000,1.000000,0.522907,0.061933,3.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23.0,16,0,100.000000,0.666667,0.032650,0.050207,2.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29.0,22,0,100.000000,0.866667,0.028555,0.064129,2.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26.0,19,0,100.000000,1.000000,0.522907,0.057606,3.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33.0,26,0,100.000000,1.000000,0.079963,0.059441,3.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235790,29.0,22,0,100.000000,1.000000,0.522907,0.058739,3.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235791,28.0,21,0,100.000000,0.785714,0.028555,0.053834,2.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235792,30.0,23,0,100.000000,1.000000,0.003319,0.063093,2.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235793,50.5,36,0,28.157537,0.465116,0.000961,0.050211,3.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Feature Scaling

##### for backward

In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled_data=scaler.fit_transform(df_cleaned_backward_updated)
scaled_df_backward=pd.DataFrame(scaled_data,columns=df_cleaned_backward_updated.columns)
scaled_df_backward.head()

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,0.480000,0.62500,0.0,1.0,1.000000,1.000000,0.605576,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.266667,0.37500,0.0,1.0,0.583333,0.062440,0.363869,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.426667,0.56250,0.0,1.0,0.833333,0.054608,0.650833,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.346667,0.46875,0.0,1.0,1.000000,1.000000,0.516375,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.533333,0.68750,0.0,1.0,1.000000,0.152920,0.554213,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### for correlation

In [28]:
scaled_data1=scaler.fit_transform(df_cleaned_corr_updated)
scaled_df_corr=pd.DataFrame(scaled_data1,columns=df_cleaned_corr_updated.columns)
scaled_df_corr.head()

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,0.480000,0.62500,0.0,1.0,1.000000,1.000000,0.605576,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.266667,0.37500,0.0,1.0,0.583333,0.062440,0.363869,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.426667,0.56250,0.0,1.0,0.833333,0.054608,0.650833,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.346667,0.46875,0.0,1.0,1.000000,1.000000,0.516375,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.533333,0.68750,0.0,1.0,1.000000,0.152920,0.554213,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
